In [77]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widjets
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from time import time
from sys import stdout
#DATA = np.load('cube100x100_60.npy')
DATA = np.load('data_circle_30_1D.npy')

In [87]:
class SOMNetwork():
    def __init__(self, x, dim=10, sigma=None, learning_rate=0.1, tau1=1000, tau2=1000, dtype=tf.float32,accuracy = 100):
        self.dtype = dtype
        self._X = x
        self._data = np.random.rand(x,dim)
        self.dim = tf.constant(dim, dtype=tf.int64)
        self._learning_rate = tf.constant(learning_rate, dtype=self.dtype, name='learning_rate')
        self._sigma = tf.constant(sigma, dtype=self.dtype, name='sigma')
        self._tau1 = tf.constant(tau1, dtype=self.dtype, name='tau1')
        self._tau2 = tf.constant(tau2, dtype=self.dtype, name='tau2')
        self._accuracy = accuracy
        self._input = tf.placeholder(shape=[dim], dtype=self.dtype, name='input')
        self._num_iter = tf.placeholder(dtype=self.dtype, name='iteration')
        self._map_weights = tf.Variable(self._data,dtype=self.dtype, name='weights')
        #self.positions = tf.where(tf.fill([dim, dim], True)) #полезно
        self.positions = tf.constant(np.arange(x))
        self._distance_map = tf.constant(self.ring_closure(self._X),dtype='float32') #[0,1,2,3,2,1]
        self.SOM_MAP = 1   #сохраняем карту сюда после сессии
        
        self.FUNC_Q = tf.Variable(0,dtype=self.dtype,name = 'FUNCTIONAL_Q')
        
    def change_params(self,sigma=6, learning_rate=0.1, tau1=1000, tau2=1000):
        self._sigma = tf.constant(sigma, dtype=self.dtype, name='sigma')
        self._learning_rate = tf.constant(learning_rate, dtype=self.dtype, name='learning_rate')
        self._tau1 = tf.constant(tau1, dtype=self.dtype, name='tau1')
        self._tau2 = tf.constant(tau2, dtype=self.dtype, name='tau2')
        
        
    def _train(self):
        with tf.name_scope('cooperation') as scope:
            #Calculating BMU
            bmu_location = self._won_neuron()
            
            _current_learning_rate = tf.maximum(self._learning_rate*tf.exp(-self._num_iter/self._tau1), 1)
            _current_sigma = tf.maximum(self._sigma*tf.exp(-self._num_iter/self._tau2), 0.1)
            
            bmu_distance = tf.reshape(tf.roll(self._distance_map,shift=[bmu_location], axis=[0]),[-1])
            neighbourhood_func = tf.exp(-bmu_distance/(2*_current_sigma))
            learning_rate_matrix = tf.multiply(_current_learning_rate, neighbourhood_func)
            delta = tf.transpose(tf.transpose(self._input-self._map_weights)*learning_rate_matrix)
            
            new_weights = tf.add(self._map_weights, delta)
            training_op = tf.assign(self._map_weights, new_weights)
        return training_op
    
    def _won_neuron(self):
        distance = tf.sqrt(tf.reduce_sum(tf.square(self._input - self._map_weights), axis=-1))
        bmu=tf.argmin(distance, axis=0)
        bmu_location = self.positions[bmu]
        return bmu_location
    
    def training(self, data, n_iters):
        self._sess = tf.Session()
        self._sess.run(tf.global_variables_initializer())
        
        #summary_writer = tf.summary.FileWriter('./logs/', tf.get_default_graph())
        #tf.summary.scalar('Functional_Q', self.FUNC_Q)
        #tf.summary.scalar('Functional_Z', self.FUNCTIONAL_Z())
        #summaries = tf.summary.merge_all()

        init = tf.global_variables_initializer()
        _data = tf.constant(data,dtype=self.dtype, name='data')
        _training = self._train()
        _functional = self.FUNCTIONAL_Z()
        data_len = len(data)
        temp_funq = 0
        
    
        with self._sess.as_default():
            
            for i in range(n_iters):
                n_item = np.random.randint(data_len)
                self._sess.run(_training, feed_dict={self._input: data[n_item], self._num_iter:i})
                if i % 50 == 0:
                    #self.FUNCTIONAL_Q()
                    #summary_writer.add_summary(self._sess.run(summaries), i)
                    #print(self._sess.run(self.FUNCTIONAL_Q()))
                    new_funq=self._sess.run(_functional)
                    #if abs(temp_funq-new_funq)<self._accuracy: break
                    #temp_funq = np.copy(new_funq)
                
                _to_print = f'\r Iters = {i}, funq = {new_funq}'
                stdout.write(_to_print)
            self.SOM_MAP = SOM._map_weights.eval()
            #print(new_funq)
            #print(f'took {i} iters, funq = {new_funq}')
        self._sess.close()
        return i, new_funq
            
    
    def check_for_nodes(self,data):
        def WIN_NEURON(vect):
            distances=np.linalg.norm(self.SOM_MAP-vect,axis=-1)
            index=np.argmin(distances)
            return index
        #for i in range(len(data)):
        for vect in data:
            a=WIN_NEURON(vect)
            print(a)
        
    def ring_closure(self,A):
        if A==1: return [0]
        a=np.arange(A)
        max_val = int(len(a)/2)
        max_ind=np.argwhere(a==max_val)[0,0]
        if len(a)%2==0:
            b=np.arange(1,a[max_ind])[::-1]
        else:
            b=np.arange(1,a[max_ind+1])[::-1]
        a[max_ind+1:]=b
        return a
    
    def FUNCTIONAL_Z(self):
        #ATTENTION! Working only for 1D maps
        #Functional_Z = 0
        B=tf.roll(self._map_weights, shift=1, axis=0)
        Functional_Z = tf.reduce_sum(tf.norm(tf.subtract(self._map_weights,B),axis=-1))
        #Functional_Z = tf.reduce_sum(tf.sqrt(tf.reduce_sum(tf.pow(tf.subtract(self._map_weights,B),2),1)))
        return Functional_Z
    
    def func_for_Q(self):
        bmu = self._won_neuron()
        dist = tf.norm(self._input-self._map_weights[bmu], axis=0)
        return dist
    
    
    def FUNCTIONAL_Q(self):
        FUNC = tf.Variable(0,dtype=self.dtype)
        FUNC_op = tf.assign(FUNC, FUNC+self.func_for_Q())
        with self._sess.as_default():
            self._sess.run(FUNC.initializer)
            for i in range(len(DATA)):
                self._sess.run(FUNC_op, feed_dict = {self._input:DATA[i]})
                #FUNC += self._sess.run(self.func_for_Q(),feed_dict = {self._input:DATA[i]})
            #print(self._sess.run(FUNC))
            self._sess.run(tf.assign(self.FUNC_Q, FUNC))

In [90]:
SOM = SOMNetwork(x=len(DATA),dim=len(DATA[0]),sigma=5,learning_rate=1,tau1=1000,tau2=1000,dtype=tf.float32)
#
#SOM.test()
time_start=time()
iters, funq = SOM.training(DATA, 10000)
print()
print('time taken: ',time()-time_start)
print(f'took{iters} iters, funq {funq}')

#SOM.look_for_nodes(DATA)
SOM_MAP = SOM.SOM_MAP

 Iters = 219, funq = 2.7942605018615723

 Iters = 436, funq = 3.432839870452881

 Iters = 649, funq = 3.8078224658966064

 Iters = 868, funq = 4.005135536193848

 Iters = 9999, funq = 6.2701215744018555
time taken:  8.859055757522583
took9999 iters, funq 6.2701215744018555


In [85]:
SOM.change_params(6,1,2000,1000)
time_start=time()
iters, funq = SOM.training(DATA, 6000)
print()
print('time taken: ',time()-time_start)
print(f'took{iters} iters, funq {funq}')
SOM_MAP = SOM.SOM_MAP

 Iters = 218, funq = 2.610118865966797

 Iters = 430, funq = 2.455320358276367

 Iters = 645, funq = 3.3227901458740234

 Iters = 860, funq = 3.6105518341064453

 Iters = 5999, funq = 6.2698335647583015
time taken:  5.503615856170654
took5999 iters, funq 6.269833564758301


In [91]:
@interact(i=(0,29,1))
def print_circle(i=0):
    plt.figure(figsize=(5,5))
    x = SOM_MAP[:,0]
    y = SOM_MAP[:,1]
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.plot(x,y,'o-')

interactive(children=(IntSlider(value=0, description='i', max=29), Output()), _dom_classes=('widget-interact',…

In [4]:
SOM.check_for_nodes(DATA)

13
14
14
15
16
17
18
19
21
22
23
24
25
26
26
27
27
28
29
1
3
4
5
6
7
8
9
10
12
13


In [29]:
@interact(i=(0,len(DATA)-1,1))
def test(i):
    plt.imshow(SOM_MAP[i].reshape(100,100))

interactive(children=(IntSlider(value=29, description='i', max=59), Output()), _dom_classes=('widget-interact'…

In [22]:
@interact(i=(0,len(DATA)-1,1))
def test(i):
    plt.imshow(DATA[i].reshape(100,100))

interactive(children=(IntSlider(value=29, description='i', max=59), Output()), _dom_classes=('widget-interact'…

In [81]:
def FUNCTIONAL_Z(data):
    #ATTENTION! Working only for 1D maps
    Functional_Z = 0
    for i in range(data.shape[0]):
        if i+1 == data.shape[0]:
            next_i = 0
        else:
            next_i = i+1
        b=np.sqrt(((data[i]-data[next_i])**2).sum())
        Functional_Z += b
        #print(i, f'{b:4.2f}')
        #print(f'for {Vectors[i]} and {Vectors[next_i]} FUNC = {b.sum()}')
        
    return Functional_Z
a=FUNCTIONAL_Z(DATA)
print(a)

6.271707796059209


In [ ]:
def _iterator():
    for learn_rate in [1]:
        for sigma in np.linspace(10, 30, 6):
            for tau1 in np.linspace(100, 5000, 10):
                for tau2 in np.linspace(100, 1500, 3):
                    yield (np.array([learn_rate, sigma, tau1, tau2]))
        
def worker(x):
    
    learn_rate, sigma, tau1, tau2 = x
    SOM.change_params(sigma,learn_rate,tau1, tau2)
    iters, funq = SOM.training(DATA, 5000)
    
    res = [learn_rate, sigma, tau1, tau2, iters, funq]
    #print(f'{res}, time {Time_exec}')
    return res




SOM = SOMNetwork(x=len(DATA),dim=len(DATA[0]),sigma=2,learning_rate=1,tau1=2000,tau2=100,dtype=tf.float32)
testing_params = []
for num_iter,x in enumerate(_iterator()):
    time_start = time()
    res = worker(x)
    Time_exec = time()-time_start
    print(f'iter {num_iter}, res {res},time {Time_exec}')
    testing_params.append(res)
    if num_iter %10==0:
        np.save('testing_params2.npy',np.array(testing_params))
np.save('testing_params2.npy',np.array(testing_params))

[1.0, 6.0, 100.0, 100.0, 2200, 179137.22], time 3.4513089656829834
iter 0, res [1.0, 6.0, 100.0, 100.0, 2200, 179137.22]
[1.0, 6.0, 100.0, 255.55555555555554, 2550, 166083.14], time 3.9888715744018555
iter 1, res [1.0, 6.0, 100.0, 255.55555555555554, 2550, 166083.14]
[1.0, 6.0, 100.0, 411.1111111111111, 3000, 150735.31], time 4.633668661117554
iter 2, res [1.0, 6.0, 100.0, 411.1111111111111, 3000, 150735.31]
[1.0, 6.0, 100.0, 566.6666666666666, 350, 24008.688], time 1.5239360332489014
iter 3, res [1.0, 6.0, 100.0, 566.6666666666666, 350, 24008.688]
[1.0, 6.0, 100.0, 722.2222222222222, 500, 30295.273], time 1.8056755065917969
iter 4, res [1.0, 6.0, 100.0, 722.2222222222222, 500, 30295.273]
[1.0, 6.0, 100.0, 877.7777777777777, 4250, 143013.88], time 6.426876068115234
iter 5, res [1.0, 6.0, 100.0, 877.7777777777777, 4250, 143013.88]
[1.0, 6.0, 100.0, 1033.3333333333333, 750, 32728.238], time 2.6140336990356445
iter 6, res [1.0, 6.0, 100.0, 1033.3333333333333, 750, 32728.238]
[1.0, 6.0, 10

[1.0, 6.0, 1711.111111111111, 411.1111111111111, 1400, 140906.95], time 7.682531833648682
iter 52, res [1.0, 6.0, 1711.111111111111, 411.1111111111111, 1400, 140906.95]
[1.0, 6.0, 1711.111111111111, 566.6666666666666, 2050, 141195.48], time 8.58714771270752
iter 53, res [1.0, 6.0, 1711.111111111111, 566.6666666666666, 2050, 141195.48]
[1.0, 6.0, 1711.111111111111, 722.2222222222222, 450, 22357.338], time 6.747542381286621
iter 54, res [1.0, 6.0, 1711.111111111111, 722.2222222222222, 450, 22357.338]
[1.0, 6.0, 1711.111111111111, 877.7777777777777, 1100, 62792.367], time 7.628135681152344
iter 55, res [1.0, 6.0, 1711.111111111111, 877.7777777777777, 1100, 62792.367]
[1.0, 6.0, 1711.111111111111, 1033.3333333333333, 350, 11278.289], time 6.825812578201294
iter 56, res [1.0, 6.0, 1711.111111111111, 1033.3333333333333, 350, 11278.289]
[1.0, 6.0, 1711.111111111111, 1188.8888888888887, 4550, 141294.22], time 12.011553764343262
iter 57, res [1.0, 6.0, 1711.111111111111, 1188.8888888888887, 455

[1.0, 7.555555555555555, 100.0, 566.6666666666666, 3550, 146342.77], time 15.326171398162842
iter 103, res [1.0, 7.555555555555555, 100.0, 566.6666666666666, 3550, 146342.77]
[1.0, 7.555555555555555, 100.0, 722.2222222222222, 400, 16313.359], time 11.633037328720093
iter 104, res [1.0, 7.555555555555555, 100.0, 722.2222222222222, 400, 16313.359]
[1.0, 7.555555555555555, 100.0, 877.7777777777777, 400, 12534.668], time 11.77364706993103
iter 105, res [1.0, 7.555555555555555, 100.0, 877.7777777777777, 400, 12534.668]
[1.0, 7.555555555555555, 100.0, 1033.3333333333333, 4850, 143647.55], time 17.19072985649109
iter 106, res [1.0, 7.555555555555555, 100.0, 1033.3333333333333, 4850, 143647.55]
[1.0, 7.555555555555555, 100.0, 1188.8888888888887, 1350, 39329.957], time 13.072648763656616
iter 107, res [1.0, 7.555555555555555, 100.0, 1188.8888888888887, 1350, 39329.957]
[1.0, 7.555555555555555, 100.0, 1344.4444444444443, 4999, 139786.44], time 17.590649604797363
iter 108, res [1.0, 7.55555555555

[1.0, 7.555555555555555, 1388.888888888889, 1033.3333333333333, 4800, 142754.94], time 21.173608541488647
iter 146, res [1.0, 7.555555555555555, 1388.888888888889, 1033.3333333333333, 4800, 142754.94]
[1.0, 7.555555555555555, 1388.888888888889, 1188.8888888888887, 1850, 67157.34], time 17.682398557662964
iter 147, res [1.0, 7.555555555555555, 1388.888888888889, 1188.8888888888887, 1850, 67157.34]
[1.0, 7.555555555555555, 1388.888888888889, 1344.4444444444443, 350, 10154.896], time 16.001854419708252
iter 148, res [1.0, 7.555555555555555, 1388.888888888889, 1344.4444444444443, 350, 10154.896]
[1.0, 7.555555555555555, 1388.888888888889, 1500.0, 850, 21501.662], time 16.70700430870056
iter 149, res [1.0, 7.555555555555555, 1388.888888888889, 1500.0, 850, 21501.662]
[1.0, 7.555555555555555, 1711.111111111111, 100.0, 600, 147580.9], time 16.498589992523193
iter 150, res [1.0, 7.555555555555555, 1711.111111111111, 100.0, 600, 147580.9]
[1.0, 7.555555555555555, 1711.111111111111, 255.55555555

[1.0, 7.555555555555555, 2677.777777777778, 1500.0, 4999, 139659.34], time 25.79728937149048
iter 189, res [1.0, 7.555555555555555, 2677.777777777778, 1500.0, 4999, 139659.34]
[1.0, 7.555555555555555, 3000.0, 100.0, 550, 160033.23], time 20.525290489196777
iter 190, res [1.0, 7.555555555555555, 3000.0, 100.0, 550, 160033.23]
[1.0, 7.555555555555555, 3000.0, 255.55555555555554, 1250, 143372.22], time 21.472233057022095
iter 191, res [1.0, 7.555555555555555, 3000.0, 255.55555555555554, 1250, 143372.22]
[1.0, 7.555555555555555, 3000.0, 411.1111111111111, 1600, 142956.42], time 22.006377935409546
iter 192, res [1.0, 7.555555555555555, 3000.0, 411.1111111111111, 1600, 142956.42]
[1.0, 7.555555555555555, 3000.0, 566.6666666666666, 2000, 140947.88], time 22.593855142593384
iter 193, res [1.0, 7.555555555555555, 3000.0, 566.6666666666666, 2000, 140947.88]
[1.0, 7.555555555555555, 3000.0, 722.2222222222222, 1000, 55968.652], time 21.50269842147827
iter 194, res [1.0, 7.555555555555555, 3000.0, 

[1.0, 9.11111111111111, 1066.6666666666667, 877.7777777777777, 4650, 142167.34], time 31.245708227157593
iter 235, res [1.0, 9.11111111111111, 1066.6666666666667, 877.7777777777777, 4650, 142167.34]
[1.0, 9.11111111111111, 1066.6666666666667, 1033.3333333333333, 4600, 141158.34], time 68.91675162315369
iter 236, res [1.0, 9.11111111111111, 1066.6666666666667, 1033.3333333333333, 4600, 141158.34]
[1.0, 9.11111111111111, 1066.6666666666667, 1188.8888888888887, 4999, 141444.6], time 35.575222969055176
iter 237, res [1.0, 9.11111111111111, 1066.6666666666667, 1188.8888888888887, 4999, 141444.6]
[1.0, 9.11111111111111, 1066.6666666666667, 1344.4444444444443, 850, 14468.596], time 31.20079278945923
iter 238, res [1.0, 9.11111111111111, 1066.6666666666667, 1344.4444444444443, 850, 14468.596]
[1.0, 9.11111111111111, 1066.6666666666667, 1500.0, 4999, 132062.5], time 37.391873836517334
iter 239, res [1.0, 9.11111111111111, 1066.6666666666667, 1500.0, 4999, 132062.5]
[1.0, 9.11111111111111, 1388.

[1.0, 9.11111111111111, 2355.5555555555557, 1344.4444444444443, 2200, 63380.848], time 34.864903688430786
iter 278, res [1.0, 9.11111111111111, 2355.5555555555557, 1344.4444444444443, 2200, 63380.848]
[1.0, 9.11111111111111, 2355.5555555555557, 1500.0, 850, 16289.432], time 68.24291253089905
iter 279, res [1.0, 9.11111111111111, 2355.5555555555557, 1500.0, 850, 16289.432]
[1.0, 9.11111111111111, 2677.777777777778, 100.0, 550, 148712.62], time 33.814428091049194
iter 280, res [1.0, 9.11111111111111, 2677.777777777778, 100.0, 550, 148712.62]
[1.0, 9.11111111111111, 2677.777777777778, 255.55555555555554, 1150, 141899.38], time 78.65823364257812
iter 281, res [1.0, 9.11111111111111, 2677.777777777778, 255.55555555555554, 1150, 141899.38]
[1.0, 9.11111111111111, 2677.777777777778, 411.1111111111111, 1700, 141729.06], time 35.111491441726685
iter 282, res [1.0, 9.11111111111111, 2677.777777777778, 411.1111111111111, 1700, 141729.06]
[1.0, 9.11111111111111, 2677.777777777778, 566.666666666666

In [16]:
a=np.load('testing_params1.npy')
ind = np.where(a[:,5]>135000)
a=a[ind]
ind = np.where(a[:,5]<150000)
a=a[ind]
ind = np.where(a[:,4]<1000)
print(ind)
print(a[ind])

(array([  2,  11,  34,  61,  73,  78,  85, 100, 108, 122, 137, 142, 149,
       156, 163, 178, 185, 192, 198, 204], dtype=int64),)
[[1.00000000e+00 6.00000000e+00 4.22222222e+02 1.00000000e+02
  8.50000000e+02 1.45321922e+05]
 [1.00000000e+00 6.00000000e+00 7.44444444e+02 1.00000000e+02
  8.50000000e+02 1.44185438e+05]
 [1.00000000e+00 6.00000000e+00 1.71111111e+03 1.00000000e+02
  7.00000000e+02 1.44024766e+05]
 [1.00000000e+00 6.00000000e+00 3.00000000e+03 1.00000000e+02
  5.50000000e+02 1.41588531e+05]
 [1.00000000e+00 7.55555556e+00 4.22222222e+02 1.00000000e+02
  8.00000000e+02 1.44922344e+05]
 [1.00000000e+00 7.55555556e+00 7.44444444e+02 1.00000000e+02
  6.00000000e+02 1.42638297e+05]
 [1.00000000e+00 7.55555556e+00 1.06666667e+03 1.00000000e+02
  7.00000000e+02 1.44515531e+05]
 [1.00000000e+00 7.55555556e+00 1.71111111e+03 1.00000000e+02
  6.00000000e+02 1.47580906e+05]
 [1.00000000e+00 7.55555556e+00 2.03333333e+03 1.00000000e+02
  5.00000000e+02 1.43369109e+05]
 [1.00000000e+

In [40]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

class SOMNetwork():
	def __init__(self, input_dim, dim=10, sigma=None, learning_rate=0.1, tay2=1000, dtype=tf.float32):
		if not sigma:
			sigma = dim / 2
		self.dtype = dtype
		#constants
		self.dim = tf.constant(dim, dtype=tf.int64)
		self.learning_rate = tf.constant(learning_rate, dtype=dtype, name='learning_rate')
		self.sigma = tf.constant(sigma, dtype=dtype, name='sigma')
		self.tay1 = tf.constant(1000/np.log(sigma), dtype=dtype, name='tay1')
		self.minsigma = tf.constant(sigma * np.exp(-1000/(1000/np.log(sigma))), dtype=dtype, name='min_sigma')
		self.tay2 = tf.constant(tay2, dtype=dtype, name='tay2')
		#input vector
		self.x = tf.placeholder(shape=[input_dim], dtype=dtype, name='input')
		#iteration number
		self.n = tf.placeholder(dtype=dtype, name='iteration')
		#variables
		self.w = tf.Variable(tf.random_uniform([dim*dim, input_dim], minval=-1, maxval=1, dtype=dtype),
			dtype=dtype, name='weights')
		#helper
		self.positions = tf.where(tf.fill([dim, dim], True))



	def training_op(self):
		win_index = self.__competition('train_')
		with tf.name_scope('cooperation') as scope:
			coop_dist = tf.sqrt(tf.reduce_sum(tf.square(tf.cast(self.positions -
				[win_index//self.dim, win_index-win_index//self.dim*self.dim], 
				dtype=self.dtype)), axis=1))
			sigma = tf.cond(self.n > 1000, lambda: self.minsigma, lambda: self.sigma * tf.exp(-self.n/self.tay1))
			sigma_summary = tf.summary.scalar('Sigma', sigma)
			tnh = tf.exp(-tf.square(coop_dist) / (2 * tf.square(sigma))) # topological neighbourhood
		with tf.name_scope('adaptation') as scope:
			lr = self.learning_rate * tf.exp(-self.n/self.tay2)
			minlr = tf.constant(0.01, dtype=self.dtype, name='min_learning_rate')
			lr = tf.cond(lr <= minlr, lambda: minlr, lambda: lr)
			lr_summary = tf.summary.scalar('Learning rate', lr)
			delta = tf.transpose(lr * tnh * tf.transpose(self.x - self.w))
			training_op = tf.assign(self.w, self.w + delta)
		return training_op, lr_summary, sigma_summary

	def __competition(self, info=''):
		with tf.name_scope(info+'competition') as scope:
			distance = tf.sqrt(tf.reduce_sum(tf.square(self.x - self.w), axis=1))
		return tf.argmin(distance, axis=0)

#== Test SOM Network ==

def test_som_with_color_data():
	som_dim = 100
	som = SOMNetwork(input_dim=3, dim=som_dim, dtype=tf.float64, sigma=3)
	test_data = np.random.uniform(0, 1, (250000, 3))
	training_op, lr_summary, sigma_summary = som.training_op()
	init = tf.global_variables_initializer()
	writer = tf.summary.FileWriter('./logs/', tf.get_default_graph())
	with tf.Session() as sess:
		init.run()
		img1 = tf.reshape(som.w, [som_dim,som_dim,-1]).eval()
		plt.figure(1)
		plt.subplot(121)
		plt.imshow(img1)
		start = time.time()
		for i, color_data in enumerate(test_data):
			if i % 1000 == 0:
				print('iter:', i)
			sess.run(training_op, feed_dict={som.x: color_data, som.n:i})
		end = time.time()
		print(end - start)
		img2 = tf.reshape(som.w, [som_dim,som_dim,-1]).eval()
		plt.subplot(122)
		plt.imshow(img2)
	writer.close()
	plt.show()		

test_som_with_color_data()

INFO:tensorflow:Summary name Learning rate is illegal; using Learning_rate instead.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


iter: 0
iter: 1000
iter: 2000


KeyboardInterrupt: 

In [2]:
def Get_Distances(y,x):
    def ring_closure(A):
        if A==1: return [0]
        a=np.arange(A)
        max_val = int(len(a)/2)
        max_ind=np.argwhere(a==max_val)[0,0]
        if len(a)%2==0:
            b=np.arange(1,a[max_ind])[::-1]
        else:
            b=np.arange(1,a[max_ind+1])[::-1]
        a[max_ind+1:]=b
        return a
    yv=ring_closure(y)
    xv=ring_closure(x)
    XV, YV = np.meshgrid(xv, yv)
    #res = np.stack((YV,XV), axis=-1)
    res = np.sqrt(XV**2+YV**2)
    return res

In [5]:
a=np.ones((x*y,5))*np.array([1,2,3,4,5])

In [22]:
x=5
y=1
def _generate_index_matrix(x, y):
        return tf.constant(np.array(list(_iterator(x, y))))
    
def _iterator(x, y):
    for i in range(x):
        for j in range(y):
            yield np.array([i, j])


            
_distance_map = tf.constant(Get_Distances(y,x),dtype='float32')
_locations = _generate_index_matrix(y, x)

# Build a dataflow graph.
_weights  = tf.Variable(a.T,dtype='float32')
_input = 3*tf.constant([1, 1, 1, 1, 1],dtype='float32')
input_matix = tf.stack([_input for i in range(x*y)])
distances = tf.sqrt(tf.reduce_sum((_weights-_input)**2, -1))
bmu = tf.argmin(distances, 0)
bmu_location = _locations[bmu]
_current_learning_rate = tf.maximum(1*tf.exp(-100/1000), 0.1)
_current_radius = 2*tf.exp(-100/1000)

bmu_distance = tf.reshape(tf.roll(_distance_map,shift=bmu_location, axis=[0, 1]),[-1])
neighbourhood_func = tf.exp(-bmu_distance/(2*_current_radius**2))
learning_rate_matrix = tf.multiply(_current_learning_rate, neighbourhood_func)
 
#Update all the weights
multiplytiplier = tf.stack([tf.tile(tf.slice(learning_rate_matrix, np.array([i]), np.array([1])), [5])
                                               for i in range(x*y)])

delta = tf.transpose(tf.transpose(_input-_weights)*learning_rate_matrix)

# Construct a `Session` to execute the graph.
#sess = tf.Session()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #print(sess.run(distances))
    #print(sess.run(_locations))
    #print(sess.run(bmu))
    print(sess.run(bmu_location))
    #print(sess.run(_distance_map))
    ##print(sess.run(bmu_distance))
    #print(sess.run(neighbourhood_func))
    print(sess.run(learning_rate_matrix))
    #print(sess.run(multiplytiplier))
    print(sess.run(delta))


# Execute the graph and store the value that `e` represents in `result`.
#result = sess.run(bmu_location)
#print(result)


[0 2]
[0.666743  0.77672   0.9048374 0.77672   0.666743 ]
[[ 1.333486  1.333486  1.333486  1.333486  1.333486]
 [ 0.77672   0.77672   0.77672   0.77672   0.77672 ]
 [ 0.        0.        0.        0.        0.      ]
 [-0.77672  -0.77672  -0.77672  -0.77672  -0.77672 ]
 [-1.333486 -1.333486 -1.333486 -1.333486 -1.333486]]


In [23]:
time_start = time()

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.multiply(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
z = tf.add(y,2, name='JEJE')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [z,x, w, y, y_, loss]:
    tf.summary.scalar(value.op.name, value)

summaries = tf.summary.merge_all()

sess = tf.Session()
summary_writer = tf.summary.FileWriter('log_simple_stats', sess.graph)

sess.run(tf.global_variables_initializer())
for i in range(100):
    summary_writer.add_summary(sess.run(summaries), i)
    #print('before step {}, y is {}'.format(i, sess.run(y)))
    #sess.run(y)
    sess.run(train_step)

print(time()-time_start)

29.147895574569702
